In [10]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [11]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [12]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
}

folder_type = ['rgb','ndvi','thumb']

In [13]:
target_folders = glob.glob(os.path.join(out_folder,'**/*'))
target_folders

['D:\\ToAWS\\MGC001\\20230811',
 'D:\\ToAWS\\MGC001\\20230816',
 'D:\\ToAWS\\MGC002\\20230823']

In [14]:
target_folder = target_folders[1]

In [15]:
course_id = course_ids.get('포항')
target_date = os.path.split(target_folder)[-1]
course_id, target_date

('MGC001', '20230816')

In [16]:
dataJson_ = glob.glob(os.path.join(out_folder,course_id,target_date,'**/photo*.json'), recursive=True)[0]
dataJson_ 

'D:\\ToAWS\\MGC001\\20230816\\photo.json'

In [17]:
with open(dataJson_, "r") as _json:
  dataJson= json.load(_json)

In [18]:
len(dataJson)

9184

In [19]:
dataJson[0]

{'id': '20230816104921_1293336709444_362275511944_156750000',
 'originalFileJPG': 'original\\2023_0816_포항CC_오전맑음오후구름가득_데이터 1차 분류본_완\\1.태백3H\\2023_0816_포항CC_태백3H_0.5m\\20230816_1039-1049\\DJI_20230816104921_0295_D.JPG',
 'originalFileGrp': ['original\\2023_0816_포항CC_오전맑음오후구름가득_데이터 1차 분류본_완\\1.태백3H\\2023_0816_포항CC_태백3H_0.5m\\20230816_1039-1049\\DJI_20230816104921_0295_MS_RE.TIF',
  'original\\2023_0816_포항CC_오전맑음오후구름가득_데이터 1차 분류본_완\\1.태백3H\\2023_0816_포항CC_태백3H_0.5m\\20230816_1039-1049\\DJI_20230816104921_0295_MS_R.TIF',
  'original\\2023_0816_포항CC_오전맑음오후구름가득_데이터 1차 분류본_완\\1.태백3H\\2023_0816_포항CC_태백3H_0.5m\\20230816_1039-1049\\DJI_20230816104921_0295_MS_NIR.TIF',
  'original\\2023_0816_포항CC_오전맑음오후구름가득_데이터 1차 분류본_완\\1.태백3H\\2023_0816_포항CC_태백3H_0.5m\\20230816_1039-1049\\DJI_20230816104921_0295_MS_G.TIF'],
 'destFolder': '20230816/1.태백3H/미기재_0',
 'GeoTagInfo': {'coords': [129.33367094444446, 36.227551194444445],
  'datetime_original': '2023:08:16 10:49:21',
  'gps_altitude': 156.75,
  'gps_alt

In [20]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [21]:
for index_, row_ in enumerate(dataJson):
  newrow = {}
  newrow ['id'] = row_['id']
  newrow ['metadata'] = course_id+target_date+row_['info']['area']+row_['info']['desc']
  newrow ['date'] = target_date
  newrow ['area'] = row_['info']['area']
  newrow ['desc'] = row_['info']['desc']
  newrow ['json'] = json.loads(json.dumps(row_), parse_float=Decimal)
  print(index_, end='\r')
  table.put_item(
    Item= newrow
  )

In [22]:
target_date

'20230816'

In [1]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

NameError: name 'table' is not defined

In [31]:
len(data)

121